# 向量存储工具包

这将帮助你快速入门 [VectorStoreToolkit](/docs/concepts/tools/#toolkits)。关于 VectorStoreToolkit 所有功能和配置的详细文档，请前往 [API 参考文档](https://api.js.langchain.com/classes/langchain.agents.VectorStoreToolkit.html)。

`VectorStoreToolkit` 是一个工具包，它接收一个向量存储，并将其转换为一个工具，该工具可以被调用，也可以传递给 LLM、代理等。

## 环境准备

如果你想从各个工具的运行中获得自动追踪信息，也可以通过取消下面的注释来设置你的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

```typescript
process.env.LANGSMITH_TRACING="true"
process.env.LANGSMITH_API_KEY="your-api-key"
```

### 安装

这个工具包位于 `langchain` 包中：

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  langchain @langchain/core
</Npm2Yarn>
```

## 实例化

现在我们可以实例化我们的工具包。首先，我们需要定义在工具包中使用的LLM。

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```

In [1]:
// @lc-docs-hide-cell

import { ChatOpenAI } from "@langchain/openai";

const llm = new ChatOpenAI({
  model: "gpt-4o-mini",
  temperature: 0,
})

In [3]:
import { VectorStoreToolkit, VectorStoreInfo } from "langchain/agents/toolkits"
import { OpenAIEmbeddings } from "@langchain/openai"
import { MemoryVectorStore } from "langchain/vectorstores/memory"
import { RecursiveCharacterTextSplitter } from "@langchain/textsplitters";
import fs from "fs";

// Load a text file to use as our data source.
const text = fs.readFileSync("../../../../../examples/state_of_the_union.txt", "utf8");

// Split the text into chunks before inserting to our store
const textSplitter = new RecursiveCharacterTextSplitter({ chunkSize: 1000 });
const docs = await textSplitter.createDocuments([text]);

const vectorStore = await MemoryVectorStore.fromDocuments(docs, new OpenAIEmbeddings());

const vectorStoreInfo: VectorStoreInfo = {
  name: "state_of_union_address",
  description: "the most recent state of the Union address",
  vectorStore,
};

const toolkit = new VectorStoreToolkit(vectorStoreInfo, llm);

## 工具

在这里，我们可以看到它将我们的向量存储转换为一个工具：

In [4]:
const tools = toolkit.getTools();

console.log(tools.map((tool) => ({
  name: tool.name,
  description: tool.description,
})))

[
  {
    name: 'state_of_union_address',
    description: 'Useful for when you need to answer questions about state_of_union_address. Whenever you need information about the most recent state of the Union address you should ALWAYS use this. Input should be a fully formed question.'
  }
]


## 在智能体中使用

首先，确保你已经安装了 LangGraph：

```{=mdx}
<Npm2Yarn>
  @langchain/langgraph
</Npm2Yarn>
```

然后，实例化智能体：

In [6]:
import { createReactAgent } from "@langchain/langgraph/prebuilt"

const agentExecutor = createReactAgent({ llm, tools });

In [8]:
const exampleQuery = "What did biden say about Ketanji Brown Jackson is the state of the union address?"

const events = await agentExecutor.stream(
  { messages: [["user", exampleQuery]]},
  { streamMode: "values", }
)

for await (const event of events) {
  const lastMsg = event.messages[event.messages.length - 1];
  if (lastMsg.tool_calls?.length) {
    console.dir(lastMsg.tool_calls, { depth: null });
  } else if (lastMsg.content) {
    console.log(lastMsg.content);
  }
}

[
  {
    name: 'state_of_union_address',
    args: {
      input: 'What did Biden say about Ketanji Brown Jackson in the State of the Union address?'
    },
    type: 'tool_call',
    id: 'call_glJSWLNrftKHa92A6j8x4jhd'
  }
]
In the State of the Union address, Biden mentioned that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson, describing her as one of the nation’s top legal minds who will continue Justice Breyer’s legacy of excellence. He highlighted her background as a former top litigator in private practice, a former federal public defender, and noted that she comes from a family of public school educators and police officers. He also pointed out that she has received a broad range of support since her nomination.
In the State of the Union address, President Biden spoke about Ketanji Brown Jackson, stating that he nominated her as one of the nation’s top legal minds who will continue Justice Breyer’s legacy of excellence. He highlighted her experience as a forme

## API 参考文档

如需了解 VectorStoreToolkit 所有功能和配置的详细文档，请访问 [API 参考文档](https://api.js.langchain.com/classes/langchain.agents.VectorStoreToolkit.html)。